# Natural Language Processing for sentiment classification (Deep Learning)

Óscar Poblete Sáenz <br> Course: Introduction to Artificial Intelligence<br>
Teacher: Elizabeth Guevara Martinez<br>
Universidad Anáhuac <br> <br>
Sources:<br>

Zhang, Ye & Wallace, Byron.  A Sensitivity Analysis of (and Practitioners' Guide to) Convolutional Neural Networks for Sentence Classification. arXiv.  2015<br>
Vishnu K https://digitaltesseract.com/sentiment-analyzer-using-convolutional-neural-network/


In [1]:
# Import libraries
import numpy as np # Matrices and vectors
import math # Math instructions
import re # Regex for string cleaning
import pandas as pd # Data analysis and manipulation
import tensorflow as tf # Tensorflow
import tensorflow_datasets as tfds # Dataset

from tensorflow.keras import layers # CNN layers
from bs4 import BeautifulSoup # Pulling data out of HTML and XML files
from google.colab import drive # Google drive

# Data

In [2]:
# Mount Google drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# Create a list ‘cols’ that holds the column names of our dataset
cols = ["sentiment", "id", "date", "query", "user", "text"]

# Store train.csv file in a variable to use it later for other steps
data = pd.read_csv(
    "/content/drive/MyDrive/Colab Notebooks/tweets_NLP/Tweets/trainTweets.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)

# Preprocessing

In [ ]:
# # Remove columns that are not required
# data.drop(["id", "date", "query", "user"], axis=1, inplace=True)
# # If the columns have already been deleted and this instruction is run again, it will mark an error.

In [4]:
# Message cleansing function
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
   # Remove the @ and its mention
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet) # r is line, + from @ to the end is taken as one
   # Remove URL links
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet) # ? makes the previous element may or may not be
   # Keeping only letters
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet) # ^ except
   # Remove extra whitespace
    tweet = re.sub(r" +", ' ', tweet) 
    return tweet

In [ ]:
# # Clean messages 
# data_clean = [clean_tweet(tweet) for tweet in data.text]

In [5]:
# 4 is used instead of 1 to denote positive sentiments. So, all the occurrences of 4 have to be replaced by 1
set(data.sentiment)
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

In [6]:
# Verify labels
set(data.sentiment)

{0, 1}

# Conversion (vectors)


In [ ]:
# # Replace each word with a vector
# tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
#     data_clean, target_vocab_size=2**16
# )

# data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

In [ ]:
# Save the encoder
# tokenizer.save_to_file('/content/drive/MyDrive/RNA/Tweets/encodeFile')

In [ ]:
# # Perform padding so all vectors are the same size
# # We take the largest word and from there the smallest words we make padding, we fill
# # cells with 0
# MAX_LEN = max([len(sentence) for sentence in data_inputs])
# data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
#                                                             value=0,
#                                                             padding="post", #Everything after that is filled with zeros.
#                                                             maxlen=MAX_LEN)

In [8]:
# Load the saved inputs
data_inputs=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_NLP/Tweets/data_inputs.csv').values

In [11]:
from sklearn.model_selection import train_test_split
# Split into training and test sets
train_inputs, test_inputs, train_labels, test_labels = train_test_split(data_inputs, data_labels, test_size=0.6)

In [ ]:
# train_inputs

# Model

In [12]:
class DCNN(tf.keras.Model):
    
    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="dcnn"):
        super(DCNN, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size,
                                          emb_dim)
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D() 
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) 
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

In [13]:
# # Define training parameters

# VOCAB_SIZE = 65540 #tokenizer.vocab_size 

# EMB_DIM = 200
# NB_FILTERS = 100
# FFN_UNITS = 256
# NB_CLASSES = 2  #len(set(train_labels))

# DROPOUT_RATE = 0.2

BATCH_SIZE = 32
# NB_EPOCHS = 1

In [ ]:
# Generate the model architecture
# Dcnn = DCNN(vocab_size=VOCAB_SIZE,
#             emb_dim=EMB_DIM,
#             nb_filters=NB_FILTERS,
#             FFN_units=FFN_UNITS,
#             nb_classes=NB_CLASSES,
#             dropout_rate=DROPOUT_RATE)

In [ ]:
# # Determine if the problem is biclass or multiclass
# if NB_CLASSES == 2:
#     Dcnn.compile(loss="binary_crossentropy", # For biclass
#                  optimizer="adam",
#                  metrics=["accuracy"])
# else:
#     Dcnn.compile(loss="sparse_categorical_crossentropy", # For multiclass
#                  optimizer="adam",
#                  metrics=["sparse_categorical_accuracy"])

In [ ]:
# # Save the training results
# checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/tweets_NLP/Tweets"

# ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

# ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if ckpt_manager.latest_checkpoint:
#     ckpt.restore(ckpt_manager.latest_checkpoint)
#     print("Last checkpoint restored!")

In [ ]:
# Class to save checkpoints
# class saveCallback(tf.keras.callbacks.Callback):
#   def on_epoch_end(self, epoch, logs=None):
#     ckpt_manager.save()
#     print("Checkpoint en {}.".format(checkpoint_path))

In [ ]:
# # Train the model
# Dcnn.fit(train_inputs,
#          train_labels,
#          batch_size=BATCH_SIZE,
#          epochs=NB_EPOCHS,
#          callbacks = [saveCallback()]
#          )

In [ ]:
# Save weights at the end of the training
# Dcnn.save_weights('/content/drive/MyDrive/Colab Notebooks/tweets_NLP/Tweets Test/weightsTweet.h5')

# Architecture and weights

In [14]:
# Initialize the parameters
VOCAB_SIZE = 65540 #tokenizer.vocab_size 

EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2  #len(set(train_labels))


In [15]:
# Establish the architecture of the ANN
Dcnn = DCNN(vocab_size= VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,training=False)

In [16]:
# Generate architecture
Dcnn.build(input_shape=(1, VOCAB_SIZE))

In [18]:
# Load weights
Dcnn.load_weights('/content/drive/MyDrive/Colab Notebooks/tweets_NLP/Tweets Test/weightsTweet.h5')

In [19]:
# Compile
# Determine if the problem is biclass or multiclass
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy", # If it is biclass then loss must be like this
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy", # If it is multiclass then loss must be like this
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

# Evaluation

In [20]:
results = Dcnn.evaluate(test_inputs, test_labels, batch_size=BATCH_SIZE)
print(results)

30000/30000 [==============================] - 91s 3ms/step - loss: 0.2387 - accuracy: 0.9094
[0.2387446165084839, 0.9094114303588867]


In [23]:
# Load the encoder
tokenizer = tfds.deprecated.text.SubwordTextEncoder.load_from_file('/content/drive/MyDrive/Colab Notebooks/tweets_NLP/Tweets Test/encodeFile')

In [24]:
# Encode text
encoded = tokenizer.encode("I hate being with you")
print(encoded)

[3, 161, 192, 26, 55]


In [25]:
# Test ANN
Dcnn(np.array([encoded]), training=False).numpy()

array([[0.00765597]], dtype=float32)

In [26]:
# Function to get the prediction
def get_prediction(sentence):
  encoded = tokenizer.encode(sentence)
  sentiment = Dcnn(np.array([encoded]), training=False).numpy()
  if sentiment  < 0.5:
    print("Negative sentiment: {} ".format(sentiment))
  else:
    print("Positive sentiment: {} ".format(sentiment))

In [27]:
# See the RNA prediction
get_prediction("I hate being with you")

Negative sentiment: [[0.00765597]] 
